## TVB TimeSeries Widget

---

### This notebook is dedicated to showcasing the TimeSeries widget using TVB data

### A. Generate data with command line interface and plot it

### Intial setup

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
from tvbwidgets.api import TimeSeriesWidget
from IPython.core.display_functions import display

### Generate TVB data

 #### 1 --  This TS has only 1 state variable and 1 mode:

In [ ]:
from tvbwidgets.tests.ts_generator import generate_ts_with_stimulus
from tvb.simulator.lab import connectivity
# tsr1 = generate_ts_with_stimulus(conn=connectivity.Connectivity.from_file())
# tsr1

#### 2 -- Generate TVB TS data with more modes:

In [ ]:
from tvbwidgets.tests.ts_generator import generate_ts_with_mode_and_sv
tsr2 = generate_ts_with_mode_and_sv(conn=connectivity.Connectivity.from_file())
tsr2

### Plot data using our TS widget

In [ ]:
tsw = TimeSeriesWidget()
tsw.add_datatype(tsr2)
display(tsw)

### B. Loading data from Drive with GUI

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
from tvbwidgets.api import TimeSeriesWidget, TimeSeriesBrowser
from IPython.core.display_functions import display

tsb = TimeSeriesBrowser()
display(tsb)